## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


# __Importing Pandas/Numpy/Zipfile__

In [2]:
# importing pandas, numpy and zipfile
import zipfile
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import json

# _Cleaning TV movies budget_